# (5.2) Solve AFAPE for dataset created on ground truth data (no missingness) 
Use different estimators to compute E[C|do(R_bar = 1)]. Also give valid confidence intervals. 

In [1]:
%load_ext autoreload
%autoreload 2

## Define paths

Paths for data

In [2]:
from afa.configurations.utils_static import specify_default_paths_static
# which dataset to work on 
dataset_name   = "synthetic_1"

# automatically specify some path locations (change paths manually if needed) (WATCH OUT: LOAD ORIGINAL DATASET) 
paths = specify_default_paths_static(dataset_name = dataset_name , miss_scenario = None) 

In [3]:
# name for agent (and predictor) 
agent_name            = 'DQN'
predictor_model_name  = 'logistic_regression'

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# new (where to save the model) 
data_dir_models = '../../../data/static/' + dataset_name + '/' + miss_scenario + '/' # where are the models 
agent_dir           = data_dir_models  + 'afa_agents' + '/' + agent_name + '/'
predictor_model_dir = data_dir_models  + 'predictor_models' + '/' + predictor_model_name + '/'

# how to name the afa_dataset
afa_dataset_name = 'ground_truth'

Define estimators

In [4]:
# define estimators 
estimator_params_list = [
        {'name': 'Ground_truth',
        'estimator_type' : 'simple_blocking' }, 
]

## Load afa dataset

In [5]:
from afa.data_modelling.datasets.data_loader.data_loader_static import DataLoader_static
from afa.afa_datasets.afa_data_loader.afa_data_loader_static import AFADataLoader_static

2023-02-26 18:41:47.672945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 18:41:47.790744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-26 18:41:47.790764: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-26 18:41:48.441688: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
# load dataset 
data_loader = DataLoader_static( data_file                  = paths['data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'], 
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

In [7]:
#load afa_dataset
augmented_data_file = agent_dir + afa_dataset_name + '_' + 'results.hkl'
afa_agent_params = None
afa_data_loader = AFADataLoader_static(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = dataset,
                    model_params        = afa_agent_params) 
afa_dataset = afa_data_loader.load() 

2023-02-26 18:41:53.726004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 18:41:53.726146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-26 18:41:53.726211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-26 18:41:53.726271: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-26 18:41:53.726331: W tensorf

## Initialize the estimators

In [8]:
from afa.afa_models.afa_estimators.utils_static import define_afa_estimator_static

In [9]:
estimators = []
for estimator_params in estimator_params_list:
    estimator = define_afa_estimator_static(  estimator_name   = estimator_params['name'] ,
                                              estimator_type   = estimator_params['estimator_type'] ,
                                              estimator_params = estimator_params) 
    estimators.append(estimator)

## Compute estimates 

In [13]:
J_bootstraps = afa_dataset.estimate_counterfactual_cost( estimators = estimators, 
                                                         fold = 0, split = "val", 
                                                         n_samples = 1, 
                                                         n_bootstraps = 10, 
                                                         n_max = None)

> /mnt/c/Users/henrik.vonkleist/Nextcloud/PhD/Code/Active Feature Acquisition/afa_ts/afa/afa_models/afa_estimators/utils.py(123)bootstrap_ids()
    121     pdb.set_trace()
    122 
--> 123     inds_bootstrapped_list = [    
    124         np.where(np.isin(ids, unique_ids_subsample[i, bootstrap_inds[i, :]]))[0]
    125         for i in range(n_bootstraps)



ipdb>  bootstrap_inds


array([[33, 21, 15, 33, 20,  3, 13, 26, 11, 39, 33,  6, 21, 28, 25, 22,
         5,  2, 16, 17, 30, 35, 22,  3, 23, 36, 34, 27, 22, 10,  4,  9,
         8, 21, 34,  9, 33, 33, 35,  3],
       [23,  9, 27, 11, 15, 33,  2, 13, 22, 16, 38, 24, 15, 34, 39, 25,
        10, 17, 22, 31, 37,  5, 10, 29, 30,  3, 11, 21, 22, 20, 27, 27,
        22, 28, 26, 29, 39, 34, 28, 21],
       [22, 22, 33, 38, 26, 15,  6, 31,  8, 35, 30,  3, 25,  0,  6,  2,
        37, 18, 20, 36, 38,  7, 14, 12,  3,  2, 31,  7, 37, 11, 35,  6,
         7, 11, 37, 12,  0,  4,  0, 19],
       [24, 20, 14,  9, 20, 17, 38, 16, 39, 23, 21, 15,  2,  0,  5, 25,
         6,  2, 27, 16,  2,  8, 27,  8, 28, 28, 18, 22, 18, 12, 31, 34,
        24, 15, 35, 34, 30, 11, 12, 28],
       [36, 34, 27, 21, 38, 11, 22,  2,  2, 38, 21, 28,  0, 39, 18, 22,
        27, 33, 27, 21, 12, 16,  6, 30,  2, 15, 15, 14, 13, 34, 30,  9,
        38, 28, 24,  2, 19,  6, 34, 21],
       [ 6,  1, 27, 18, 17, 17, 36,  1, 27, 26,  4, 30, 15, 27, 32, 19,
   

ipdb>  bootstrap_inds[0]


array([33, 21, 15, 33, 20,  3, 13, 26, 11, 39, 33,  6, 21, 28, 25, 22,  5,
        2, 16, 17, 30, 35, 22,  3, 23, 36, 34, 27, 22, 10,  4,  9,  8, 21,
       34,  9, 33, 33, 35,  3])


...

ipdb>  unique_ids_subsample


array([[89, 19,  5, 34, 57, 85, 98, 48, 68, 87, 33, 42, 99, 44, 24, 58,
        22, 26, 49, 55, 47, 13, 90, 38, 84, 77, 92, 93, 82, 88, 83, 94,
        54, 27, 95,  0, 20, 11, 71, 15],
       [33, 54, 38, 26, 82, 34, 44, 95, 48, 57, 77, 19, 83, 87, 13, 94,
        15, 27, 11, 24, 89,  5, 99, 20, 58, 71, 85, 47, 22, 68, 98, 84,
        42, 93,  0, 92, 55, 90, 88, 49],
       [68, 77, 99, 98, 13, 82, 38, 58, 44, 92, 57, 93, 24, 84, 85, 33,
        19, 22, 11, 47, 89, 83, 48, 94, 87, 20, 26, 27, 49, 71, 90, 88,
         0, 55, 54, 34, 42, 15, 95,  5],
       [84, 87, 93, 68, 11, 54, 57, 83, 82, 22, 94, 42,  5, 88, 26, 98,
        55, 44, 19, 47, 99, 90, 13, 85, 92,  0, 33, 49, 24, 48, 34, 27,
        20, 95, 15, 89, 77, 58, 38, 71],
       [99, 95, 49, 22, 13, 89, 87, 47, 93, 77, 58, 85,  5, 88, 82,  0,
        42, 15, 68, 27, 11, 26, 54, 48, 84, 20, 55, 38, 34, 19, 71, 24,
        57, 44, 90, 98, 83, 94, 33, 92],
       [27, 48, 22, 47, 99, 90, 85, 71, 34, 92,  5, 83, 57, 58, 89, 54,
   

ipdb>  unique_ids_subsample[0]


array([89, 19,  5, 34, 57, 85, 98, 48, 68, 87, 33, 42, 99, 44, 24, 58, 22,
       26, 49, 55, 47, 13, 90, 38, 84, 77, 92, 93, 82, 88, 83, 94, 54, 27,
       95,  0, 20, 11, 71, 15])


ipdb>  exit


In [11]:
J_bootstraps 

{'Ground_truth': array([1.51851852, 1.27272727, 1.18      , 1.15384615, 0.73076923,
        1.61363636, 1.5       , 1.125     , 1.46296296, 0.77272727])}

In [ ]:
# save estimate
from afa.afa_models.afa_estimators.utils import save_results_bootstrapping
save_results_bootstrapping( J_bootstraps , agent_dir, afa_dataset_name = afa_dataset_name )

## Compute estimates for convergence
If we know the ground truth, we might be interesting in plotting convergence, for this we might want to compute estimates J for different amount of available datapoints. 

In [ ]:
from afa.afa_models.afa_estimators.utils import compute_counterfactual_cost_convergence

In [ ]:
J_bootstraps_convergence, convergence_steps  = compute_counterfactual_cost_convergence(  afa_dataset = afa_dataset, 
                                                                                         estimators = estimators, 
                                                                                         fold = 0, split = "val", 
                                                                                         n_samples = 1, 
                                                                                         n_bootstraps = 10)

In [ ]:
# save estimators
save_results_bootstrapping( J_bootstraps_convergence , agent_dir, convergence_steps = convergence_steps, afa_dataset_name = afa_dataset_name)